In [ ]:
%matplotlib inline
import pandas as pd
import subprocess
import itertools
import numpy as np

# Configuracion
command = "./tp {input} {output} {method}"

tests = [
    {'input': 'experimento1-1',
     'output': 'experimento1-1-outputs/{method}-{h}.test',
     'method': [0, 1],
     'skip': 1,
     'h': [0.5, 0.8, 1.0, 1.25, 2.0, 2.5, 4.0, 5.0, 6.25, 10.0, 12.5]},
    {'input': 'experimento1-2',
     'output': 'experimento1-2-outputs/{method}-{h}.test',
     'method': [0, 1],
     'skip': 1,
     'h': [0.5, 0.8, 1.0, 1.25, 2.0, 2.5, 4.0, 5.0, 6.25, 10.0, 12.5]},
    {'input': 'experimento1-3',
     'output': 'experimento1-3-outputs/{method}-{h}.test',
     'skip': 1,
     'method': [0, 1],
     'h': [0.5, 0.8, 1.0, 1.25, 2.0, 2.5, 4.0, 5.0, 6.25, 10.0, 12.5]},
    {'input': 'experimento2-1',
     'output': 'experimento2-1-outputs/{name}-{method}-{h}.test',
     'method': [0, 1],
     'runs': 5,
     'skip': 1,
     'h': [0.5, 0.8, 1.0, 1.25, 2.0, 2.5, 4.0, 5.0, 6.25, 10.0, 12.5]},
    {'input': 'experimento3-1',
     'inputs': ['experimento3-1-0', 'experimento3-1-1', 'experimento3-1-2', 'experimento3-1-3', 'experimento3-1-4', 'experimento3-1-5', 'experimento3-1-6', 'experimento3-1-7', 'experimento3-1-8', 'experimento3-1-9', 'experimento3-1-10'],
     'output': 'experimento3-1-outputs/{name}-{method}-{h}.test',
     'method': [2, 3],
     'h': [1]}]

# Generacion de datos
for file in tests:
    name = file['input']
    
    dataset = []
    
    try:
        if file['skip']:
            print 'Salteando experimento', name
            continue
    except:
        pass
    
    print 'Comenzando experimento', name
    
    try:
        names = file['inputs']
    except:
        names = [name]
        
    for nameI in names:
        print 'Corriendo subexperimento', nameI
        
        with open(nameI + '.input', 'r') as handle:
            file['contents'] = handle.read()

            for (run_method, run_h) in itertools.product(file['method'], file['h']):
                output = {}
                output['name'] = nameI
                output['method'] = run_method
                output['h'] = run_h

                with open('/tmp/tmp.test', 'w+') as writer:
                    writer.write(file['contents'].format(h=run_h))

                outFile = file['output'].format(name=nameI, h=run_h, method=run_method)
                
                try:
                    runs = file['runs']
                except:
                    runs = 1
                
                for r in xrange(runs):
                    fmt = command.format(input='/tmp/tmp.test', output=outFile, method=run_method)
                    runtime = subprocess.check_output(fmt, shell=True)

                    for line in runtime.split('\n'):
                        if line:
                            line = line.split(':')
                            line[0] = line[0].strip().lower()
                            line[1] = float(line[1].strip())
                            output[line[0]] = line[1]

                    dataset.append(output)

    dataset = pd.DataFrame(dataset)
    dataset.to_csv(name + '.csv', index=False)
    print 'CSV guardado'
        
    print 'Experimento', name, 'finalizado'

Salteando experimento experimento1-1
Salteando experimento experimento1-2
Salteando experimento experimento1-3
Salteando experimento experimento2-1
Comenzando experimento experimento3-1
Corriendo subexperimento experimento3-1-0
Corriendo subexperimento

In [ ]:
# Experimento 1-1
# Muchas sanguijuelas pequeñas cerca del punto crítico, la idea es que a medida que aumentemos la granularidad
# van a empezar a aparecer en la discretización, haciendo que la temperatura del punto crítico vaya tornándose
# cada vez más alta.
import numpy.random as npr

n=50
width=100
height=100

minX = width/2 - 5
maxX = width/2 + 5
minY = height/2 - 5
maxY = height/2 + 5

posX = npr.uniform(minX, maxX, n)
posY = npr.uniform(minY, maxY, n)
radius = map(abs, npr.normal(0.5, max([maxX - minX, maxY - minY]), n))
temperatures = npr.exponential(100, n)

output = zip(posX, posY, radius, temperatures)

print width, height, "{h}", n
for (x, y, r, t) in output:
    print x, y, r, t

In [ ]:
# Experimento 1-2
# Sanguijuelas alejadas, de un radio comparativamente grande.
# Lo que debería suceder en este caso es que no se modificaría la temperatura del punto crítico a medida
# que incrementemos la granularidad.
import numpy.random as npr

n=50
width=100
height=100

posX = npr.uniform(0, width, n)
posY = npr.uniform(0, height, n)
radius = map(abs, npr.normal(2, max(width, height), n))
temperatures = npr.exponential(100, n)

output = zip(posX, posY, radius, temperatures)

print width, height, "{h}", n
for (x, y, r, t) in output:
    print x, y, r, t

In [ ]:
# Experimento 1-3
# Tomar sanguijuelas pequeñas, alejadas del centro, con temperaturas muy altas.
# Si las temperaturas son lo suficientemente altas se deberían propagar hacia el centro a medida que aumente h
import numpy.random as npr

n=50
width=100
height=100

posX = npr.uniform(0, width, n)
posY = npr.uniform(0, height, n)
radius = map(abs, npr.normal(0.1, max(width, height)/10, n))
temperatures = npr.exponential(300, n)

output = zip(posX, posY, radius, temperatures)

print width, height, "{h}", n
for (x, y, r, t) in output:
    print x, y, r, t

In [ ]:
# Experimento 2-1
# Este experimento es solo para ver que pasa con las sanguis
import numpy.random as npr

n=50
width=100
height=100

posX = npr.uniform(0, 10, n)
posY = npr.uniform(0, 10, n)
radius = npr.uniform(0, 10, n)
temperatures = npr.exponential(300, n)

output = zip(posX, posY, radius, temperatures)

print width, height, "{h}", n
for (x, y, r, t) in output:
    print x, y, r, t

In [23]:
# Experimento 3-1
# Tomamos una grilla de sanguijuelas situadas en los puntos de la discretización (no variamos el h)
# y vamos aumentando la cantidad de sanguijuelas unitarias (es decir, las que tienen radio < h).

# IMPORTANTE: Si se cambia el h, cambiarlo en el primer casillero también
import numpy as np
import numpy.random as npr
import copy
import itertools

width=100
height=100
h = 1
percentages = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
outputs = ['experimento3-1-0', 'experimento3-1-1', 'experimento3-1-2', 'experimento3-1-3', 'experimento3-1-4', 'experimento3-1-5', 'experimento3-1-6', 'experimento3-1-7', 'experimento3-1-8', 'experimento3-1-9', 'experimento3-1-10']

posX = list(np.arange(0, width, h))
posY = list(np.arange(0, height, h))

# Aca comienza el codigo

n = 101
radiusLower = list(npr.uniform(0.2, h - 0.2, n - 1))
radiusHigher = list(npr.uniform(h + 0.1, h + 0.4, n - 1))
temperatures = list(npr.exponential(300, n - 1))
positions = list(itertools.product(posX, posY))

for (fname, x) in zip(outputs, percentages):
    with open(fname + '.input', 'w+') as handle:        
        print >> handle, width, height, "{h}", n
        leftPositions = copy.deepcopy(positions)
        leftRL = copy.deepcopy(radiusLower)
        leftRH = copy.deepcopy(radiusHigher)
        leftT = copy.deepcopy(temperatures)

        noUnitarias = round(x*float(n - 1))
        unitarias = float(n - 1) - noUnitarias
        
        t = noUnitarias
        noUnitarias = unitarias
        unitarias = t
        print unitarias, noUnitarias

        i = 0
        while i < unitarias:
            # Generamos la cantidad de sanguijuelas unitarias que hagan falta
            (x, y) = leftPositions.pop()
            r = leftRL.pop()
            t = leftT.pop()

            print >> handle, x, y, r, t
            i += 1.0

        i = 0
        while i < noUnitarias:
            # El resto son sanguijuelas no unitarias
            (x, y) = leftPositions.pop()
            r = leftRH.pop()
            t = leftT.pop()

            print >> handle, x, y, r, t
            i += 1.0
        
        print >> handle, (width/2)/h, (height/2)/h, h - 0.4354, 1000.0

0.0 100.0
10.0 90.0
20.0 80.0
30.0 70.0
40.0 60.0
50.0 50.0
60.0 40.0
70.0 30.0
80.0 20.0
90.0 10.0
100.0 0.0


In [ ]:
# Selección de los h para la experimentación, queremos ver qué h sirven con el width y height que buscamos.
# Si hay más de 50 000 de dims, va a ser complicado que el experimento corra bien, asi que el h tiene que
# estar por debajo de eso para que la cosa ande.
width=100.0
height=100.0

lst = [x for x in np.arange(0.01, 100, 0.01) if round(width/x) == width/x and round(height/x) == height/x]
[(x, (width/x)*(height/x)) for x in lst]

In [ ]:
map(lambda (x, y): x, [(0.5, 40000.0), (0.80000000000000004, 15625.0), (1.0, 10000.0), (1.25, 6400.0), (2.0, 2500.0), (2.5, 1600.0), (4.0, 625.0), (5.0, 400.0), (6.25, 256.0), (10.0, 100.0), (12.5, 64.0), (50.0, 4.0)])

In [ ]:
list(np.arange(0, 100, 1))

In [20]:
dataset

,cpt,delta,h,method,name,removed_leech,slp
0,58.0998,0.309673,1,2,experimento3-1-0,-1,0.009990
1,58.0998,0.593068,1,3,experimento3-1-0,-1,0.009990
2,58.0998,0.315646,1,2,experimento3-1-1,-1,0.107892
3,58.0998,0.593654,1,3,experimento3-1-1,-1,0.107892
4,58.0998,0.313538,1,2,experimento3-1-2,-1,0.205794
5,58.0998,0.595916,1,3,experimento3-1-2,-1,0.205794
6,58.0998,0.319098,1,2,experimento3-1-3,-1,0.303696
7,58.0998,0.597111,1,3,experimento3-1-3,-1,0.303696
8,58.0998,0.312909,1,2,experimento3-1-4,-1,0.401598
9,58.0998,0.592500,1,3,experimento3-1-4,-1,0.401598
